#### **EMBEDDINGS CREATION WITH W2V**

- i'm going to use the subset created in the previows notebook
- this subset will be used for create the embeddings with W2V
- every time you install **gensim** if you have trouble restart the kernel

In [1]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install gensim

In [2]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from tqdm import tqdm
from google.colab import files

tqdm.pandas()

In [3]:
#UPLOAD TRAINING AND TEST:

train_path = "/content/drive/MyDrive/DatasetHLT/VV1/train_sampled.csv"
test_path = "/content/drive/MyDrive/DatasetHLT/VV1/test_sampled.csv"

In [4]:
# LET'S SEE IF IT'S CORRECTLY UPLOADED:

df_train_sampled = pd.read_csv(train_path)
df_test_sampled = pd.read_csv(test_path)

df_train_sampled.head(10)

,Text,Rating
0,no lonesome dove lonesome dive one good book I...,3
1,not hope do not expect magicali pretty easy to...,2
2,roof bag bag hold good however not waterproof ...,2
3,suspensful truly read review write previous re...,5
4,receive broken son receive gift bowl break tel...,2
5,computer literate little bit not read may alre...,1
6,not sequel iron monkey ! run mill hong kong ac...,2
7,nice read read many singer bio come across sim...,4
8,cheaply make buy digital camera vacation embar...,1
9,work ps ok let get straight yes work ps becuas...,5


In [5]:
#TOKENIZZAZIONE TRAINING SET:
sentences = df_train_sampled['Text'].apply(lambda x: str(x).split()).tolist()
print(sentences[0])
print(sentences[0][0])

['no', 'lonesome', 'dove', 'lonesome', 'dive', 'one', 'good', 'book', 'I', 've', 'ever', 'read', 'dead', 'man', 'walk', 'can', 'not', 'hold', 'candle', 'ranger', 'portray', 'bumble', 'idiot', 'capture', 'ambush', 'easily', 'pick', 'baby', 'especially', 'do', 'not', 'like', 'end', 'thought', 'call', 'gus', 'would', 'buffalo', 'hump', 'not', 'will', 'not', 'spoil', 'end', 'still', 'good', 'read', 'no', 'lonesome', 'dove', 'street', 'laredo', 'excellent', 'read', '!', 'find', 'garage', 'sale', 'thrift', 'store', 'cheap', 'price', 'mean', 'pick']
no


#### **Word 2 Vec**
- Need to understand how to set the hyperparameters int he best way

In [6]:
#HYPERPARAMETERS TO SET: HERE NEED THE GUYS FROM AI

w2v_model = Word2Vec(
    sentences,
    vector_size=100,
    window=5,
    min_count=5,
    workers=4,
    sg=1,
    seed=42
)

# 6. Funzione per generare vettore medio di embedding
def get_sentence_vector(sentence, model, vector_size):
    words = str(sentence).split()
    valid_words = [word for word in words if word in model.wv]
    if not valid_words:
        return np.zeros(vector_size)
    return np.mean(model.wv[valid_words], axis=0)

vector_size = w2v_model.vector_size

# 7. Embedding training
train_vectors = df_train_sampled['Text'].progress_apply(
    lambda x: get_sentence_vector(x, w2v_model, vector_size)
)

dataset_train_embeddings = pd.DataFrame(train_vectors.tolist())
dataset_train_embeddings['Rating'] = df_train_sampled['Rating'].values

# 8. Embedding test
test_vectors = df_test_sampled['Text'].progress_apply(
    lambda x: get_sentence_vector(x, w2v_model, vector_size)
)
dataset_test_embeddings = pd.DataFrame(test_vectors.tolist())
dataset_test_embeddings['Rating'] = df_test_sampled['Rating'].values

# 9. Save on Google Drive
train_out_path = "/content/drive/MyDrive/DatasetHLT/VV1/dataset_train_embeddings.csv"
test_out_path = "/content/drive/MyDrive/DatasetHLT/VV1/dataset_test_embeddings.csv"

dataset_train_embeddings.to_csv(train_out_path, index=False)
dataset_test_embeddings.to_csv(test_out_path, index=False)

# 10. (Opzionale) Scarica localmente
#files.download(train_out_path)
#files.download(test_out_path)

100%|██████████| 30000/30000 [00:04<00:00, 6822.83it/s]
